In [ ]:
import pandas as pd
import numpy as np

C_olv_mol_wt = np.array([60.084, 79.866, 101.961, 71.846, 70.9374, 40.304, 56.079, 61.979, 94.2, 283.889])

data = pd.read_csv('Mafic Enclave Major Element data Mt. Shasta.csv')

data.columns = data.columns.str.strip()

data = data.drop(columns=['SAMPLE', 'Sum'])

for sample_name, row in data.iterrows():
    C_og_initial = np.array([
        row['SiO2'], row['TiO2'], row['Al2O3'], row['FeO*'], row['MnO'],
        row['MgO'], row['CaO'], row['Na2O'], row['K2O'], row['P2O5']
    ])

    C_og_initial = np.nan_to_num(C_og_initial)

    C_s = (C_og_initial * C_olv_mol_wt) / ((np.sum(C_olv_mol_wt) + np.sum(C_og_initial)) * 100)
    Fo_n = 0
    iterations = 0

    while Fo_n < 90:
        iterations += 1
        X_olv_melt = 1 / (((0.32 * (C_s[3] / C_s[5])) + 1) * (3 / 2))
        Fo_n = (3 / 2) * 100 * X_olv_melt
        X_SiO2 = 1.0 / 3.0
        X_MgO = (Fo_n / 100) * (2.0 / 3.0)
        X_FeO = (2.0 / 3.0) - X_MgO
        C_olv_new = np.array([X_SiO2, 0, 0, X_FeO, 0, X_MgO, 0, 0, 0, 0])
        C_prenorm = (C_olv_new * 0.01) + (C_s * 0.99)
        C_norm = C_prenorm / np.sum(C_prenorm)
        C_s = (C_og_initial * C_norm) / (100 * (np.sum(C_norm) + np.sum(C_og_initial)))

    print(f"Sample {sample_name}: Final Fo_num reached {Fo_n} after {iterations} iterations. Final Formula: {C_s}")
